In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Pandas app\\End-to-end-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Pandas app\\End-to-end-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    colsample_bytree: float
    learning_rate: float
    max_depth: int
    n_estimators: int
    subsample: float
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params=self.params.XGBRegressor
        schema=self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            colsample_bytree= params.colsample_bytree,
            learning_rate= params.learning_rate,
            max_depth= params.max_depth,
            n_estimators= params.n_estimators,
            subsample= params.subsample,
            target_column=schema.name
        )
        return model_trainer_config

In [8]:
import pandas as pd
import os
from mlProject import logger
import xgboost as xgb
import joblib

In [9]:
class ModelTrainer:
    def __init__(self,config) -> None:
        self.config=config
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        
        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        model = xgb.XGBRegressor(colsample_bytree=self.config.colsample_bytree,learning_rate=self.config.learning_rate,max_depth=self.config.max_depth,n_estimators=self.config.n_estimators,subsample=self.config.subsample,random_state=42)
        
        model.fit(X_train,y_train)
        joblib.dump(model,os.path.join(self.config.root_dir, self.config.model_name))
        

In [10]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

2024-09-25 12:54:41,725 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-25 12:54:41,725 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-25 12:54:41,730 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-25 12:54:41,734 : INFO : common : created directory at: artifacts
2024-09-25 12:54:41,734 : INFO : common : created directory at: artifacts/model_trainer
